<a href="https://colab.research.google.com/github/derrodo61/notebooks/blob/main/neo4j_create.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Basic cypher commands

## Create nodes using MERGE

```
# Note that when you use MERGE to create a node, you must specify
# at least one property that will be the unique primary key for the node.

MERGE (p:Person {name: 'Michael Caine'})

# multiple MERGE claused in 1 code block

MERGE (p:Person {name: 'Katie Holmes'})
MERGE (m:Movie {title: 'The Dark Knight'})
RETURN p, m
```

```
LOAD CSV WITH HEADERS
from 'https://raw.githubusercontent.com/MidnightSkyUniverse/udemyNeo4j/master/data/titles_and_skills.csv'
AS row
MERGE (t:Title {name: row.Title})
```

```
LOAD CSV WITH HEADERS
from 'https://raw.githubusercontent.com/MidnightSkyUniverse/udemyNeo4j/master/data/skills_details.csv'
AS row
MERGE (s:Skill {name: row.Skill})
SET
s.id = row.ID,
s.description = row.Description,
s.category=row.Category
```

## Create nodes using CREATE

CREATE does not look up the primary key before adding the node -> faster than MERGE.

```
CREATE (n:Person {name: 'Alice', age: 30})
RETURN n
```

## `DELETE` nodes

```
MATCH (p:Person)
WHERE p.name = 'Jane Doe'
DELETE p
```

## Add property

### Inline as part of the `merge` clause

```
MATCH (p:Person {name: 'Michael Caine'})
MERGE (m:Movie {title: 'Batman Begins'})
MERGE (p)-[:ACTED_IN {roles: ['Alfred Penny']}]->(m)
RETURN p,m
```

### Using `SET``

```
MATCH (p:Person)-[r:ACTED_IN]->(m:Movie)
WHERE p.name = 'Michael Caine' AND m.title = 'The Dark Knight'
SET r.roles = ['Alfred Penny']
RETURN p, r, m

# multiple properties

MATCH (p:Person)-[r:ACTED_IN]->(m:Movie)
WHERE p.name = 'Michael Caine' AND m.title = 'The Dark Knight'
SET r.roles = ['Alfred Penny'], m.released = 2008
RETURN p, r, m

# csv import

LOAD CSV WITH HEADERS
from 'https://raw.githubusercontent.com/MidnightSkyUniverse/udemyNeo4j/master/data/titles_and_skills.csv'
AS row
MATCH (t:Title {name: row.Title})
SET t.skills = split(row.Skills, '|')
```

## Remove property

by using the REMOVE keyword, or setting the property to null.

You should never remove the property that is used as the primary key for a node.

```
# from nodes

MATCH (t:Title)
REMOVE t.skills

MATCH (p:Person)
WHERE p.name = 'Gene Hackman'
SET p.born = null
RETURN p

# from relations

MATCH (p:Person)-[r:ACTED_IN]->(m:Movie)
WHERE p.name = 'Michael Caine' AND m.title = 'The Dark Knight'
REMOVE r.roles
RETURN p, r, m

```

# Update property

```
MATCH (p:Person)-[r:ACTED_IN]->(m:Movie)
WHERE p.name = 'Michael Caine' AND m.title = 'The Dark Knight'
SET r.roles = ['Mr. Alfred Penny']
RETURN p, r, m
```

## Create edges / relations

By default, if you do not specify the direction when you create the relationship, it will always be assumed left-to-right.

```
MATCH (p:Person {name: 'Michael Caine'})
MATCH (m:Movie {title: 'The Dark Knight'})
MERGE (p)-[:ACTED_IN]->(m)

# check
MATCH (p:Person {name: 'Michael Caine'})-[:ACTED_IN]-(m:Movie {title: 'The Dark Knight'})
RETURN p, m

# This code successfully creates the nodes and relationship:
MERGE (p:Person {name: 'Emily Blunt'})-[:ACTED_IN]->(m:Movie {title: 'A Quiet Place'})
RETURN p, m

# step by step

// Find or create a person with this name
MERGE (p:Person {name: 'Michael Caine'})

// Find or create a movie with this title
MERGE (m:Movie {title: 'The Cider House Rules'})

// Find or create a relationship between the two nodes
MERGE (p)-[:ACTED_IN]->(m)

# special case using UNWIND

MATCH (t:Title)
UNWIND t.skills AS skill
MERGE (s:Skill {name: skill})
MERGE (t)-[:REQUIRES]->(s)
```

## Add additional labels

```
MATCH (s:Skill{category:'Certification'})
SET s:Skill:Certification
RETURN s.name, labels(s)

MATCH (s:Skill{category:'SpecializedSkill'})
SET s:Skill:SpecializedSkill
RETURN s.name, labels(s)

MATCH (s:Skill) REMOVE s.category
```

## Delete a label

```
MATCH (p:Person {name: 'Jane Doe'})
REMOVE p:Developer
RETURN p
```

## Delete relationships (and nodes)

```
# deletes a relationship

MATCH (p:Person {name: 'Jane Doe'})-[r:ACTED_IN]->(m:Movie {title: 'The Matrix'})
DELETE r
RETURN p, m

# deletes a node and all its relationships

MATCH (p:Person {name: 'Jane Doe'})
DETACH DELETE p
```

## Match

```
MATCH (n) RETURN n

MATCH (n:Skill|Title) RETURN n

MATCH (n:Title) RETURN n LIMIT 25

MATCH (t:Title) RETURN t.name AS name, t.skills AS skills LIMIT 5

MATCH (s:Skill) RETURN DISTINCT s.category

MATCH (n) RETURN LABELs(n) as labels, count(*) as

MATCH (s:Skill) WHERE s.name = '.NET' RETURN s.name

# case sensitive!
MATCH (s:Skill) WHERE s.name CONTAINS ('Oracle') RETURN s

MATCH (t)-[r:REQUIRES]-(s)
RETURN t,r,s

MATCH (t:Title)-[r:REQUIRES]-(s:Skill)
RETURN t,r,s

PROFILE MATCH (t)-[r:REQUIRES]-(s)
RETURN t,r,s

MATCH (t)-[r:REQUIRES]->(s:Skill)
RETURN DISTINCT LABELS(s), count(*)

# get the number of directors of the movie 'Cloud Atlas'
MATCH (m:Movie {title: 'Cloud Atlas'})<-[:DIRECTED]-(p)  RETURN COUNT(p) as num

PROFILE MATCH (t)-[r:REQUIRES]-(s)
RETURN t,r,s

PROFILE MATCH (t:Title)-[r:REQUIRES]-(s:Skill)
RETURN t,r,s

#Filtering

MATCH (p:Person)-[:ACTED_IN]->(m:Movie)
WHERE m.released = 2008 OR m.released = 2009
RETURN p, m

MATCH (p)-[:ACTED_IN]->(m)
WHERE p:Person AND m:Movie AND m.title='The Matrix'
RETURN p.name

MATCH (p:Person)-[:ACTED_IN]->(m:Movie)
WHERE 2000 <= m.released <= 2003
RETURN p.name, m.title, m.released

MATCH (p:Person)-[:ACTED_IN]->(m:Movie)
WHERE p.name='Jack Nicholson' AND m.tagline IS NOT NULL
RETURN m.title, m.tagline

# Cypher has a set of string-related keywords that you can
# use in your WHERE clauses to test string property values.
# You can specify STARTS WITH, ENDS WITH, and CONTAINS.

MATCH (p:Person)-[:ACTED_IN]->()
WHERE p.name STARTS WITH 'Michael'
RETURN p.name

MATCH (p:Person)-[:ACTED_IN]->()
WHERE toLower(p.name) STARTS WITH 'michael'
RETURN p.name

# exclude (specific relationships): exists

MATCH (p:Person)-[:WROTE]->(m:Movie)
WHERE NOT exists( (p)-[:DIRECTED]->(m) )
RETURN p.name, m.title

# lists (all list elements have the same type)

MATCH (p:Person)
WHERE p.born IN [1965, 1970, 1975]
RETURN p.name, p.born

# check if 'Neo' is in the list: IN

MATCH (p:Person)-[r:ACTED_IN]->(m:Movie)
WHERE  'Neo' IN r.roles AND m.title='The Matrix'
RETURN p.name, r.roles

# return the keys (property names) of nodes

MATCH (p:Person)
RETURN p.name, keys(p)

# show all properties of a graph

CALL db.propertyKeys()



```

## Customizing `MERGE` behavior


If you want to set multiple properties for an ON CREATE SET or ON MATCH SET clause, you separate them by commas.

```
// Find or create a person with this name
MERGE (p:Person {name: 'McKenna Grace'})

// Only set the `createdAt` property if the node is created during this query
ON CREATE SET p.createdAt = datetime()

// Only set the `updatedAt` property if the node was created previously
ON MATCH SET p.updatedAt = datetime()

// Set the `born` property regardless
SET p.born = 2006

RETURN p
```

## Schema procedures

```
call db.schema.visualization
call db.schema.nodeTypeProperties
call db.schema.relTypeProperties
```

## Vector index

Create ‘embedding’ property.

APOC has to be installed: https://neo4j.com/docs/apoc/current/installation/

```
LOAD CSV WITH HEADERS
FROM 'https://raw.githubusercontent.com/MidnightSkyUniverse/udemyNeo4j/master/data/embeddings.csv'
AS row
MATCH (s:Skill {name: row.Skill})
CALL db.create.setNodeVectorProperty(s,'embedding',apoc.convert.fromJsonList(row.Embedding))
RETURN count(*)
```

List Skill nodes and property ‘embedding’

```
MATCH (s:Skill) RETURN s.name,
s.embedding
```

Create a vector index with
‘SkillDescription’ as its name, add it to
‘Skill’ node, use ‘embedding’ property
to search through, set 384* as vector
length, use cosine similarity as search
function.

(*) it’s important to know vector length
that you have created. OpenAI vector
is 1536 long. For the purpose of the
project we used Sentence Transformer
from LangChain which generates
vectors 384 long

```
CALL db.index.vector.createNodeIndex(
'skillDescription',
'Skill',
'embedding',
384,
'cosine'
)
```

Show vector index

```
SHOW INDEXES YIELD id, name, type, state,
populationPercent WHERE type = "VECTOR"
```

## Show all labels

```
CALL db.labels()
```

## Refactoring / Using specific realtionships

Create new relationships based on properties:

```
MATCH (n:Actor)-[:ACTED_IN]->(m:Movie)
CALL apoc.merge.relationship(n,
  'ACTED_IN_' + left(m.released,4),
  {},
  {},
  m ,
  {}
) YIELD rel
RETURN count(*) AS `Number of relationships merged`;
```